In [1]:
import cPickle as pickle
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import wordnet as wn


In [2]:
wnl = WordNetLemmatizer()

In [3]:
with open('./data/imagenet_top1k_captions.pkl', 'rb') as f:
    captions = pickle.load(f)
with open('./data/top1k.pkl', 'rb') as f:
    top1k = dict(pickle.load(f))
    


In [4]:
errPerWord = {}
for k,v in captions.iteritems():
    w = top1k[k]
    wl = wnl.lemmatize(w)
    print w, wl
    errors = sum([1 for cap in v if wl not in map(wnl.lemmatize, nltk.word_tokenize(cap))])
    print errors, len(v)

woman woman
523 1405
street street
706 1297
person person
1209 1242
people people
462 1431
white white
963 1243


In [5]:
captions.keys()

[u'n10787470', u'n04334599', u'n00007846', u'n07942152', u'n09638875']

In [6]:
for k in captions:
    print top1k[k]

woman
street
person
people
white


In [7]:
captions[u'n00007846']

[u'a man and a woman are posing for a picture .',
 u'a woman is looking at her cell phone .',
 u'a man riding a bike down a street with a dog on its back .',
 u'a man in a red shirt is playing a game with a wii controller .',
 u'a man in a red shirt and tie standing in front of a christmas tree .',
 u'a man with a hat and a hat on his head .',
 u'a young woman wearing a hat and a hat .',
 u'a man in a suit and tie is sitting on a couch .',
 u'a man is standing in front of a mirror .',
 u'a woman standing next to a man in a suit .',
 u'a young woman wearing a white shirt and a red tie .',
 u'a man standing in front of a kitchen counter with a dog .',
 u'a group of people standing in a store .',
 u'a man is sitting on a bench in the woods .',
 u'a man in a suit and tie with a red tie .',
 u'a woman is sitting on a bench with her arms crossed .',
 u'a man with a tie and a tie .',
 u'a person is riding a skateboard on a ramp .',
 u'a man standing on a sidewalk next to a fire hydrant .',
 u

In [21]:
  wn.synsets('person')
print person
a = wn.synsets('person')[0].hyponyms()
print person.lemmas()
# print [str(lemma.name) for lemma in person.lemmas]

IndexError: list index out of range

In [ ]:
start = datetime.now()
    
caption_file = 'data/annotations/captions_train2014.json'
image_dir = 'image/train2014_resized'
max_length = 15
word_count_threshold = 1
vgg_model_path = './data/imagenet-vgg-verydeep-19.mat'
batch_size = 50

print '1. Building Top 1K dictionary from Train dataset'

if not os.path.exists('./data/top1k.pkl'):
    train_dataset = _process_caption_data(caption_file=caption_file,
                                      image_dir=image_dir,
                                      max_length=max_length)
    word_to_idx = _build_vocab(annotations=train_dataset, threshold=word_count_threshold)
    save_pickle(word_to_idx, './data/word_to_idx.pkl')
    top1k = _build_top1k_vocab(train_dataset)
    save_pickle(top1k, './data/top1k.pkl')
else:
    top1k = load_pickle('./data/top1k.pkl')